In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -P data/tinyshakespeare/

--2024-03-14 14:50:16--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘data/tinyshakespeare/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-03-14 14:50:16 (8.89 MB/s) - ‘data/tinyshakespeare/input.txt’ saved [1115394/1115394]



In [6]:
with open('data/tinyshakespeare/input.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

In [9]:
print(len(text))

1115394


In [8]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [18]:
dictionary = sorted(list(set(text)))
vocab_size = len(dictionary)
print(''.join(dictionary))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [27]:
stoi = {s:i for i,s in enumerate(dictionary)}
itos = {i:s for i,s in enumerate(dictionary)}
encode = lambda s: [stoi[x] for x in s]
decode = lambda l: ''.join(itos[x] for x in l)
print(encode('Kicior to szczur'))
print(decode(encode('Kicior to szczur')))

[23, 47, 41, 47, 53, 56, 1, 58, 53, 1, 57, 64, 41, 64, 59, 56]
Kicior to szczur


In [34]:
import torch
data = encode(text)
tensor = torch.tensor(data, dtype=int64)
print(tensor.shape, tensor.dtype)
print(tensor[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
